<a href="https://colab.research.google.com/github/Song-yiJung/DH-Buddhist-Analysis-Tutorial/blob/main/6_%ED%95%9C%EB%AC%B8%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%A1%9C_Word2Vec%EB%AA%A8%EB%8D%B8_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **📜 단어에 '의미'를 부여하는 Word2Vec**

TF-IDF는 문서 내에서 통계적으로 '중요한' 단어를 추출하는 지만, **단어의 '의미'를 모른다**는 결정적인 한계를 가진다.

TF-IDF에게 왕(王)과 임금은 그저 철자가 다른, 완전히 별개의 단어이다. 즉, 유학(儒學)과 성리학(性理學)의 관계를 전혀 알지 못한다.

**Word2Vec**은 이러한 한계를 뛰어넘기 위해, "단어의 의미 자체를 숫자로 표현할 수 없을까?"라는 고민에서 출발한 대표적인 **예측 기반 임베딩(Prediction-based Embedding)**이다.

### 1. **Word2Vec**의 핵심 개념: **"문맥이 의미를 만든다"**
Word2Vec의 가장 핵심적인 철학은 언어학자 J.R. Firth의 유명한 말, "단어의 의미는 그 단어 주변에 함께 등장하는 단어들을 보면 알 수 있다 (You shall know a word by the company it keeps)" 이다.

예) 어떤 사람 'A'가 항상 '궁궐', '왕비', '신하'라는 사람들과 어울린다면, 우리는 'A'가 '왕' 또는 그와 비슷한 신분일 것이라고 추측할 수 있다.

임금이라는 단어가 문헌 속에서 왕, 궁궐, 옥새, 신하 등의 단어와 자주 함께 등장한다면, 컴퓨터는 임금의 의미가 이 주변 단어들과 밀접한 관련이 있다고 학습한다.

Word2Vec은 이 아이디어를 수학적으로 구현한 것이다. 각 단어를 **수백 차원의 좌표(벡터)**를 가진 공간 속의 한 점으로 표현한다. 이 과정에서 비슷한 문맥을 가진 단어들은 '의미 공간'에서 서로 가까운 곳에 위치하게 된다.

### 2. 등장 배경: 왜 Word2Vec이 필요했나?

* 누가/언제: Word2Vec은 2013년, 구글의 토마스 미콜로프(Tomas Mikolov) 연구팀에 의해 개발되었다.

* 왜: 2010년대 초반, 딥러닝 기술이 부상하고 처리할 수 있는 데이터의 양이 폭발적으로 증가하면서, 기존의 통계 기반 모델(BoW, TF-IDF)의 한계가 명확해졌다. 더 나은 기계 번역, 감성 분석 등을 위해서는 컴퓨터가 단어의 '의미'와 '관계'를 이해하는 것이 필수적이었다. Word2Vec은 방대한 텍스트로부터 단어의 의미 벡터를 매우 효율적으로 학습할 수 있는 방법을 제시하며 자연어 처리 분야에 혁명을 가져왔다.

* 어떻게 의미를 학습하는가? (CBOW와 Skip-gram)
Word2Vec은 인공신경망을 이용해 두 가지 방식으로 '추측 게임'을 하며 단어를 학습한다.

 1) CBOW (Continuous Bag-of-Words): 주변 단어로 중심 단어 맞추기 (빈칸 채우기)

 인공신경망에게 "왕은 [ ? ]에게 명을 내렸다." 와 같은 문장을 주고, 주변 단어('왕은', '에게')를 통해 빈칸에 들어갈 단어 '신하'를 예측하도록 훈련시킨다.

 2) Skip-gram: 중심 단어로 주변 단어 맞추기 (이웃 추측하기)

 인공신경망에게 '신하'라는 단어를 주고, 그 주변에 등장할 확률이 높은 단어들('왕은', '에게')을 예측하도록 훈련시킨다.

이런 식의 예측 훈련을 수억, 수십억 번 반복하면서, 비슷한 예측 게임에 등장하는 단어들은 자연스럽게 서로 유사한 벡터 값을 갖게 된다.

## 📜 Word2Vec 실습: '추측 게임'으로 단어의 의미를 학습하는 과정

🎯 실습 목표

1. 아주 작은 문서 묶음(코퍼스)을 직접 만들어 본다.

2. 동일한 데이터로 **CBOW**와 **Skip-gram** 모델을 각각 훈련시킨다.

3. 훈련된 모델에게 "이 단어와 비슷한 의미의 단어는 뭐야?"라고 질문하여, 컴퓨터가 문맥을 통해 단어의 유사성을 정말 학습했는지 확인한다.

### ⚙️ 전체 실습 코드
아래 코드를 복사하여 Colab이나 로컬 환경에서 직접 실행한다.

In [ ]:
# 1. 필요 라이브러리 불러오기

!pip install gensim
from gensim.models import Word2Vec

# 2. 실습용 데이터 준비 (작은 문서 묶음)
# 왕, 여왕, 신하, 백성 등의 단어가 비슷한 문맥에 등장하도록 구성
corpus = [
    '왕은 신하에게 명을 내렸다',
    '여왕은 백성에게 명을 내렸다',
    '왕은 백성을 사랑했다',
    '여왕은 신하를 신뢰했다'
]

# Word2Vec 모델은 '토큰화된 문장'들의 리스트를 입력으로 받습니다.
# 문장을 띄어쓰기 기준으로 잘라 리스트로 만들어 줍니다.
tokenized_corpus = [sentence.split() for sentence in corpus]
# 결과: [['왕은', '신하에게', ...], ['여왕은', '백성에게', ...], ...]

print("토큰화된 데이터:", tokenized_corpus)
print("-" * 50)


# 3. Word2Vec 모델 훈련시키기

# 3-1. CBOW 모델 훈련 (sg=0)
# '주변 단어로 중심 단어 맞추기' 방식으로 학습
model_cbow = Word2Vec(sentences=tokenized_corpus, sg=0, vector_size=10, window=2, min_count=1)

# 3-2. Skip-gram 모델 훈련 (sg=1)
# '중심 단어로 주변 단어 맞추기' 방식으로 학습
model_sg = Word2Vec(sentences=tokenized_corpus, sg=1, vector_size=10, window=2, min_count=1)


# 4. 결과 확인하기: 단어 유사도 검증

print("▼▽▼▽ 훈련된 모델 성능 확인 ▼▽▼▽\n")

# CBOW 모델에게 '왕은'과 가장 유사한 단어를 물어보기
similar_words_cbow = model_cbow.wv.most_similar('왕은')
print(f"CBOW 모델이 찾은 '왕은'과 유사한 단어: {similar_words_cbow}")

# Skip-gram 모델에게 '왕은'과 가장 유사한 단어를 물어보기
similar_words_sg = model_sg.wv.most_similar('왕은')
print(f"Skip-gram 모델이 찾은 '왕은'과 유사한 단어: {similar_words_sg}")

print("-" * 50)

# CBOW 모델에게 '신하에게'와 가장 유사한 단어를 물어보기
similar_words_cbow_subject = model_cbow.wv.most_similar('신하에게')
print(f"CBOW 모델이 찾은 '신하에게'와 유사한 단어: {similar_words_cbow_subject}")

# Skip-gram 모델에게 '신하에게'와 가장 유사한 단어를 물어보기
similar_words_sg_subject = model_sg.wv.most_similar('신하에게')
print(f"Skip-gram 모델이 찾은 '신하에게'와 유사한 단어: {similar_words_sg_subject}")

토큰화된 데이터: [['왕은', '신하에게', '명을', '내렸다'], ['여왕은', '백성에게', '명을', '내렸다'], ['왕은', '백성을', '사랑했다'], ['여왕은', '신하를', '신뢰했다']]
--------------------------------------------------
▼▽▼▽ 훈련된 모델 성능 확인 ▼▽▼▽

CBOW 모델이 찾은 '왕은'과 유사한 단어: [('백성을', 0.2941223680973053), ('백성에게', 0.20713211596012115), ('여왕은', 0.10494355112314224), ('신하를', 0.09267307072877884), ('명을', -0.1055101752281189), ('사랑했다', -0.11387499421834946), ('내렸다', -0.21133744716644287), ('신하에게', -0.2315719872713089), ('신뢰했다', -0.36627137660980225)]
Skip-gram 모델이 찾은 '왕은'과 유사한 단어: [('백성을', 0.2941223680973053), ('백성에게', 0.20713211596012115), ('여왕은', 0.10494355112314224), ('신하를', 0.09267307072877884), ('명을', -0.1055101752281189), ('사랑했다', -0.11387499421834946), ('내렸다', -0.21133744716644287), ('신하에게', -0.2315719872713089), ('신뢰했다', -0.36627137660980225)]
--------------------------------------------------
CBOW 모델이 찾은 '신하에게'와 유사한 단어: [('신하를', 0.2914133667945862), ('명을', 0.05541810393333435), ('신뢰했다', 0.042647670954465866), ('백성에게', -0.02176340483129024

### 📊 1. 결과에 대한 솔직한 분석: 왜 기대와 달랐을까?

먼저, 우리가 기대했던 이상적인 결과(예: '왕은'과 가장 비슷한 단어 1순위가 '여왕은')와 약간 다른 결과가 나온 점을 주목하는 것이 중요하다.

'왕은'과 가장 유사한 단어로 '백성을'이 '여왕은'보다 더 높은 점수를 받았다.

'신하에게'와 유사한 단어로 '백성에게'는 거의 관련이 없는 것(점수 -0.02)으로 나왔다.

그 이유는 **학습 데이터가 압도적으로 부족하기 때문**이다.

컴퓨터는 단 4개의 문장만을 보고 '왕', '여왕', '신하', '백성'의 의미를 학습해야 했다. 이는 마치 외국인이 한국어를 배우기 위해 책 4줄만 읽고 모든 단어의 뉘앙스를 파악하려는 것과 같다.

### 🧠 2. 개념 원리 다시 이해하기: 데이터가 부족할 때 생기는 일

Word2Vec은 텍스트에 나타난 통계적 패턴을 학습하여 단어의 의미를 '추측'한다. 우리의 작은 데이터셋에서 컴퓨터가 어떤 패턴을 발견했는지 살펴보겠다.

**학습 데이터**:

왕은 신하에게 명을 내렸다

여왕은 백성에게 명을 내렸다

왕은 백성을 사랑했다

여왕은 신하를 신뢰했다

* 왜 '왕은'의 1순위가 '백성을'일까?

 컴퓨터의 추론 과정은 다음과 같다.

1. '왕은'이라는 단어의 주변을 보니, 3번 문장 왕은 백성을 사랑했다에서 '백성을'이라는 단어와 아주 가까이 붙어 있다.

2. 반면, '여왕은'은 '왕은'과 같은 문장에 등장한 적이 단 한 번도 없습니다.

3. 단 4개의 문장만 본 컴퓨터에게는, **같은 문장에서 함께 등장했다는 사실('왕은'과 '백성을')**이, 다른 문장에서 비슷한 역할을 한다는 사실('왕은'과 '여왕은')보다 훨씬 더 강력한 통계적 증거가 된다.

4. 그래서 컴퓨터는 "아, '왕은'이라는 단어는 '백성을'이라는 단어와 매우 밀접한 관련이 있구나!"라고 결론 내리고 가장 높은 유사도 점수를 준 것이다.

5. 만약 수천, 수만 개의 문장을 학습시킨다면, '왕은'과 '여왕은'이 공통적으로 '명령하다', '다스리다', '궁궐에 살다' 등 훨씬 더 다양한 문맥을 공유하게 되면서, 우리가 기대하는 대로 '여왕은'의 유사도 점수가 가장 높아지게 될 것이다.



### 📝 3. 실제 결과 한 줄씩 뜯어보기

이제 실제 출력된 결과를 이 관점에서 자세히 해석해 보겠다.

**'왕은'과 유사한 단어 분석**:

1위 '백성을' (점수: 0.294): 위에서 설명했듯, 3번 문장에서 직접적으로 함께 등장했기 때문에 가장 높은 점수를 받았다.

3위 '여왕은' (점수: 0.104): 모델이 '왕은'과 '여왕은'이 문장의 주어 자리에 오고, 비슷한 구조(예: ... 명을 내렸다)를 가진다는 것을 어느 정도 학습했음을 보여준다. 하지만 데이터가 부족하여 그 신호가 약하게 잡힌 것이다.

음수(-) 점수들: 점수가 음수인 단어들은 이 작은 데이터 안에서 '왕은'과 거의 정반대의 문맥에 등장했음을 의미한다. 즉, '관련 없음' 또는 '의미 공간의 반대편'에 위치한 단어라고 컴퓨터가 판단한 것이다.

**'신하에게'와 유사한 단어 분석**:

1위 '신하를' (점수: 0.291): 컴퓨터가 '신하'라는 동일한 어근을 공유하고 비슷한 맥락에 쓰인다는 것을 매우 잘 학습했다. 이것은 성공적인 결과이다.

4위 '백성에게' (점수: -0.021): 점수가 0에 가깝고 심지어 음수이다. 왜일까? 우리의 데이터에서 '신하에게'는 '왕은'과 짝을 이루고, '백성에게'는 '여왕은'과 짝을 이룬다. 이 둘이 함께 등장하는 문장이 없었기 때문에, 컴퓨터는 이 두 단어의 의미적 유사성을 발견할 통계적 근거를 전혀 찾지 못한 것이다.

### ✨ 4. 결론: 디지털 인문학 연구자를 위한 교훈

이번 실습의 "기대에 못 미치는" 결과는 2가지 이해를 준다.

1. 데이터의 양과 질이 분석 결과를 결정한다: Word2Vec과 같은 예측 기반 모델의 성능은 전적으로 학습 데이터에 의존한다. 조선 시대 '왕'의 의미를 알려면, 사료 4줄이 아니라 조선왕조실록 전체를 보여줘야 하는 것과 같다. 적은 데이터로 얻은 결과는 '틀린' 것이 아니라, 그 데이터 안에서만 유효한 '편향된' 결과일 뿐이다.

2. 결과에 대한 비판적 사고의 중요성: 디지털 인문학 연구자는 컴퓨터가 뱉어낸 숫자를 맹신해서는 안 된다. 결과가 왜 이렇게 나왔는지 그 원인을 파악하고, 자신의 연구 분야 지식(Domain Knowledge)을 바탕으로 결과를 비판적으로 해석하는 능력이 필수적이다. 이번 경우, 우리는 "데이터가 부족해서"라고 명확히 원인을 진단할 수 있었다.

이 코드 실습은 Word2Vec이 어떻게 작동하는지를 보여줌과 동시에, 성공적인 디지털 인문학 연구가 컴퓨터의 계산 능력과 연구자의 깊이 있는 인문학적 통찰력의 결합을 통해 이루어짐을 명확히 보여주는 예시이다.

## 📜심화 실습: 『반야심경』과 『금강경』 비교 분석하기

아래 코드는 GitHub에서 두 경전의 XML 원문을 각각 불러와 전처리한 후, 하나의 Word2Vec 모델로 학습시켜 그 의미 관계를 비교 분석하는 전체 파이프라인이다.

In [ ]:
# 1. 필요 라이브러리 설치 및 업그레이드
!pip install --upgrade numpy scipy requests beautifulsoup4 lxml jieba gensim opencc-python-reimplemented -q

print("✅ 라이브러리 설치 및 업그레이드 완료!")
# [알림] 위 셀 실행 후 버전 충돌 오류가 발생하면, 메뉴에서 [런타임] > [런타임 다시 시작]을 한 번 실행한 뒤 아래 셀을 실행하세요.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 24.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
✅ 라이브러리 설치 및 업그레이드 완료!


In [ ]:
# --- 라이브러리 불러오기 ---
import requests
from bs4 import BeautifulSoup
import re
from opencc import OpenCC
import jieba
from gensim.models import Word2Vec

# --- 1. 데이터 확보 (반야심경, 금강경) ---
urls = {
    'heart_sutra': 'https://raw.githubusercontent.com/cbeta-org/xml-p5/master/T/T08/T08n0251.xml', # 반야심경 (T251)
    'diamond_sutra': 'https://raw.githubusercontent.com/cbeta-org/xml-p5/master/T/T08/T08n0235.xml' # 금강경 (T235)
}

documents_raw = []
print("데이터 불러오는 중...")
for key, url in urls.items():
    try:
        response = requests.get(url)
        response.raise_for_status()
        documents_raw.append(response.text)
        print(f"✅ {key} 불러오기 성공!")
    except requests.exceptions.RequestException as e:
        print(f"오류: {key}를 불러오는 중 문제가 발생했습니다. ({e})")
        documents_raw.append("") # 실패 시 빈 문자열 추가


데이터 불러오는 중...
✅ heart_sutra 불러오기 성공!
✅ diamond_sutra 불러오기 성공!


In [ ]:
# --- 2. 텍스트 추출 및 전처리 ---
def preprocess_text(xml_text):
    soup = BeautifulSoup(xml_text, 'xml')
    body = soup.find('body')
    if not body:
        return ""
    text = body.get_text()
    text = re.sub(r'\s+', ' ', text)
    return text

cc = OpenCC('t2s')
documents_simplified = [cc.convert(preprocess_text(doc)) for doc in documents_raw]
print("\n✅ 텍스트 추출 및 간체자 변환 완료!")


# --- 3. 사용자 사전 추가 및 토큰화 ---
# 두 경전의 핵심 단어를 모두 사전에 추가
important_words = [
    '般若波罗蜜多', '观自在菩萨', '舍利子', '五蕴', '空', '涅槃', '阿耨多罗三藐三菩提',
    '须菩提', '世尊', '菩萨', '我相', '人相', '众生相', '寿者相', '善男子', '善女人'
]
for word in important_words:
    jieba.add_word(word)

tokenized_corpus = [jieba.lcut(doc) for doc in documents_simplified]
print("✅ 토큰화 완료!")

In [ ]:
# --- 4. Word2Vec 모델 훈련 ---
# 두 경전의 텍스트를 함께 학습하여 하나의 의미 공간을 만듭니다.
model = Word2Vec(sentences=tokenized_corpus, sg=1, vector_size=150, window=10, min_count=2, epochs=30, seed=42)
print("✅ Word2Vec 모델 훈련 완료!")


# --- 5. 비교 분석 결과 탐색 ---
print("\n--- 🚀 『반야심경』 vs 『금강경』 비교 분석 ---")

# 질문 1: 두 경전의 공통 사상인 '반야'와 관련된 단어는?
print("\n[질문 1] '般若波罗蜜多'(반야바라밀다)와 가장 유사한 단어는?")
try:
    similar_to_prajna = model.wv.most_similar('般若波罗蜜多', topn=3)
    print("▶️ 결과:", similar_to_prajna)
except KeyError:
    print("▶️ 결과: 모델 어휘 사전에 '般若波罗蜜多'가 없습니다.")

# 질문 2: 각 경전의 고유한 인물을 모델이 구분하는가?
print("\n[질문 2-1] '舍利子'(사리자)와 유사한 단어는? (주로 반야심경의 맥락)")
try:
    similar_to_sariputra = model.wv.most_similar('舍利子', topn=3)
    print("▶️ 결과:", similar_to_sariputra)
except KeyError:
    print("▶️ 결과: 모델 어휘 사전에 '舍利子'가 없습니다.")

print("\n[질문 2-2] '须菩提'(수보리)와 유사한 단어는? (주로 금강경의 맥락)")
try:
    similar_to_subhuti = model.wv.most_similar('须菩提', topn=3)
    print("▶️ 결과:", similar_to_subhuti)
except KeyError:
    print("▶️ 결과: 모델 어휘 사전에 '须菩提'가 없습니다.")

# 질문 3: 의미 연산을 통해 '금강경'의 핵심 개념을 추론할 수 있는가?
print("\n[질문 3] '菩萨' + '须菩提' - '舍利子' = ?")
print("         ('보살'의 개념에서 '사리자'의 문맥을 빼고, '수보리'의 문맥을 더하면?)")
try:
    # 금강경의 핵심 개념인 '상(相)'에 대한 집착을 버리는 것과 관련된 단어가 나올 것으로 기대
    analogy = model.wv.most_similar(positive=['菩萨', '须菩提'], negative=['舍利子'], topn=3)
    print("▶️ 결과:", analogy)
except KeyError as e:
    print(f"▶️ 결과: '{e.args[0]}' 단어가 어휘 사전에 없어 연산이 불가능합니다.")

## 📊 1. 결과 분석: 컴퓨터는 '의미'가 아닌 '쓰임새'를 본다

* [질문 1] '般若波羅蜜多'(반야바라밀다)와 유사한 단어: 依, 罣碍, 苦

**결과 해석**:

모델은 '반야바라밀다'와 비슷한 다른 철학 용어를 찾아내지 않았다. 대신, 『반야심경』의 핵심 구절인 "보살은 반야바라밀다를 의지하므로(依般若波羅蜜多故), 마음에 걸림이 없고(...心無罣碍),... 온갖 괴로움을 없앤다(能除一切苦)" 라는 문장 구조를 통째로 학습했다.

**인문학적 의미**:

컴퓨터는 '반야바라밀다'를 **'의지하여(依) 마음의 걸림(罣碍)과 괴로움(苦)을 해결하는 기능적 주체'**로 파악한 것이다. 즉, 개념의 정의가 아닌, 텍스트 내에서의 **역할과 기능(function)**을 정확히 포착했다.

* [질문 2-1] '舍利子'(사리자)와 유사한 단어: 识, 苦, 尽

**결과 해석**:

이 또한 문맥 학습의 결과이다. 『반야심경』은 "사리자여! 물질이 공과 다르지 않다..."라고 부른 뒤, "느낌, 생각, 지어감, 의식(受想行識)도 그러하니라", "온갖 괴로움과 재앙(一切苦厄)을 건넜다", "무명이 다함(無無明盡)까지도 없다" 와 같이, '사리자'를 부른 직후에 특정 개념들을 나열하거나 부정하는 패턴을 반복한다.

**인문학적 의미**:

모델은 '사리자'를 '진리를 설명하기 위해 호명되는 대상'으로 인식하고, 그 주변에 자주 함께 등장하는 단어들(識, 苦, 盡)을 유사 단어로 제시했다. '사리자'의 인격적 의미보다는 경전 내에서의 문법적, 구조적 역할을 학습한 것이다.

* [질문 2-2] '須菩提'(수보리)와 유사한 단어: ！, 「, ？

**결과 해석**:

모델이 '수보리'와 가장 유사하다고 찾아낸 것은 다른 인물이나 개념이 아닌 각종 문장 부호이다. 왜일까요? 『금강경』은 부처님과 수보리의 **'대화'**로 이루어져 있다.

* 須菩提！ (수보리야!) -> 감탄 부호 ！

* 「...」 -> 인용 및 대화 부호

* ...何以故？ (~까닭은 무엇인가?) -> 물음표 ？

**인문학적 의미**:

이것은 분석의 실패라고 보기 어렵다. Word2Vec은 '수보리'라는 단어가 텍스트 내에서 '대화의 시작과 끝, 질문과 감탄'을 이끄는 문법적 표지(marker) 역할을 한다는 사실을 정확하게 간파한 것이다. 즉, 텍스트의 장르(대화체)와 구조적 특징을 완벽하게 학습했다.

## 💡 2. 디지털 인문학적 활용 방안
이 결과를 통해 우리는 다음과 같은 심화 연구의 가능성을 엿볼 수 있다.

1. 단어의 '기능적 역할' 분석

TF-IDF가 '무엇'에 대한 글인지를 알려준다면, Word2Vec은 특정 단어가 텍스트 안에서 '어떻게' 사용되는지를 보여준다. '반야바라밀다'가 다른 철학 용어가 아닌 '의지하다', '괴로움'과 연결된 것은, 이 경전에서 반야바라밀다가 추상적 개념이 아니라 **괴로움을 해결하는 실천적 수단(機能)**으로 강조됨을 의미한다. 이러한 기능적 분석은 기존의 주석적 연구를 보완하는 새로운 관점을 제공할 수 있다.

2. 문체 및 장르의 계량적 식별

'수보리'가 문장 부호와 가깝게 나온 결과는, 우리가 이 모델을 문체나 장르를 식별하는 도구로 사용할 수 있음을 시사한다. 예를 들어, 수많은 텍스트를 학습시킨 모델에게 특정 단어(예: 왕의 이름)를 질문했을 때, 유독 문장 부호나 특정 접속사가 높은 유사도를 보인다면 "아, 이 왕에 대한 기록은 주로 대화체나 논쟁적 형식으로 남아있구나"라는 새로운 연구 가설을 세울 수 있다.

3. 의미 벡터 연산의 해석 (菩萨 + 须菩提 - 舍利子)
결과: ！(감탄), 不受(받아들이지 않음), 护念(보호하고 기억함)


해석: 이 결과는 "일반적인 '보살'의 개념에서, 『반야심경』의 '사리자'가 등장하는 문맥(철학적 요약 및 부정의 나열)을 빼고, 『금강경』의 '수보리'가 등장하는 문맥(대화, 가르침, 질문)을 더했더니, **『금강경』에서 보살에게 요구되는 핵심적인 태도(不受, 护念)와 그 경전의 문체(！)**가 남았다"고 해석할 수 있다. 이는 모델이 두 경전의 미묘한 관점 차이를 성공적으로 계산해냈음을 의미한다.


**최종 결론**:

Word2Vec은 우리에게 '사전적 정답'을 알려주는 것이 아니라, 텍스트 자체의 내적 논리와 구조, 단어의 실제 쓰임새를 거울처럼 비춰준다. 때로는 예상치 못한 결과(예: 수보리와 ！)야말로, 인간 연구자가 미처 주목하지 못했던 텍스트의 고유한 특징을 발견하게 하는 가장 중요한 단서가 될 수 있다.

**실습해보자!**

In [ ]:
# 1. 필요 라이브러리 불러오기
from gensim.models import Word2Vec

# 2. 원문 준비: 고승전 1권(ABC, K1074 v32, p.764b01)
text = '''
來。會彼學徒留礙，蘭乃閒行而至。旣達雒陽，與騰同止，少時便善漢言。
愔於西域獲經，卽爲翻譯『十地斷結』、『佛本生』、『法海藏』、『佛本行』、『四十二章』等五部。
移都寇亂，四部失本，不傳江左。唯四十二章經今見在，可二千餘言。
漢地見存諸經，唯此爲始也。
愔又於西域得畫釋迦倚像，是優田王栴檀像師第四作也。旣至雒陽，明帝卽令畫工圖寫。
置淸涼臺中，及顯節陵上。舊像今不復存焉。
又昔漢武穿昆明池底得黑灰，以問東方朔，朔云：“不委，可問西域人。”
後法蘭旣至，衆人追以問之，蘭云：“世界終盡，劫火洞燒，此灰是也。”朔言有徵，信者甚衆。
蘭後卒於雒陽，春秋六十餘矣。
安淸，字世高，安息國王正后之太子也。
幼以孝行見稱，加又志業聰敏，剋意好學，外國典籍及七曜五行醫方異術，乃至鳥獸之聲，無不綜達。
嘗行見群燕，忽謂伴曰：“燕云應有送食者。”頃之果有致焉。
衆咸奇之，故儁異之聲，早被西域。
高雖在居家，而奉戒精
'''

# 3. 전처리: 문장 단위 분리 및 자단위 토큰화
import re

# 문장 나누기 (句讀點과 마침표 기준)
sentences = re.split(r'[。！？]', text)
sentences = [s.strip() for s in sentences if s.strip()]

# 자 단위 토큰화 (한문은 의미 단위가 한 글자인 경우 많음)
tokenized_sentences = [[char for char in sentence] for sentence in sentences]

print("토큰화된 문장 수:", len(tokenized_sentences))
print("예시:", tokenized_sentences[0])

# 4. Word2Vec 모델 훈련
# 4-1. CBOW 모델 (sg=0)
model_cbow = Word2Vec(sentences=tokenized_sentences, sg=0, vector_size=50, window=3, min_count=1)

# 4-2. Skip-gram 모델 (sg=1)
model_sg = Word2Vec(sentences=tokenized_sentences, sg=1, vector_size=50, window=3, min_count=1)

# 5. 유사도 확인
print("\n▼ CBOW 모델 유사 단어 ('蘭'):")
print(model_cbow.wv.most_similar('蘭'))

print("\n▼ Skip-gram 모델 유사 단어 ('蘭'):")
print(model_sg.wv.most_similar('蘭'))


의미적 유사성 평가
‘蘭’이 회색 ‘灰’와 유사하다고 판단한 것은 문맥 상 함께 등장한 문장이 있었기 때문. 실제 본문에서 다음과 같은 문장이 존재: “後法蘭旣至，衆人追以問之，蘭云：‘世界終盡，劫火洞燒，此灰是也.’”
→ 여기서 ‘蘭’과 ‘灰’는 같은 문장 안에서 가까이 등장하므로 높은 유사도는 Word2Vec의 학습 메커니즘상 정당한 결과.

현재 문제점
1. 학습 데이터가 매우 적어 의미 기반 분류는 신뢰도 낮음.
2. 자 단위 토큰화로 인해 복합 개념(예: 佛本生, 四十二章 등)이 쪼개져 의미 손실.

개선 방법
1. 데이터 확장: 『고승전』 전권 또는 유사 한문 자료 포함.
2. 어절 단위 토큰화: “佛本生” 같은 복합어는 1개의 토큰으로 유지해야 의미 학습 가능. 사전 기반 N-gram 토큰화 또는 의미 단위 정제?
3. 불용자 제거: 조사나 접속 부사 역할의 글자(如 ‘之’, ‘焉’, ‘也’) 제거 필요.

In [ ]:
# 1. 필요 라이브러리 불러오기
from gensim.models import Word2Vec
import re

# 2. 해심밀경(解深蜜經) 제1권(ABC, K0154 v10, p.709a01)
text = '''
一時，薄伽梵住最勝光曜七寶莊嚴，放大光明，普照一切無邊世界，無量方所妙飾閒列，
周圓無際，其量難測，超過三界所行之處，勝出世閒善根所起，最極自在淨識爲相。
如來所都，諸大菩薩衆所雲集，無量天、龍、藥叉、健達縛、阿素洛、揭路茶、
緊捺洛、牟呼洛伽、人、非人等，常所翼從，廣大法味喜樂所持，作諸衆生一切義利，
滅諸煩惱災撗纏垢，遠離衆魔、過諸莊嚴，如來莊嚴之所依處，大念、慧、行以爲遊路，
大止妙觀以爲所乘，大空、無相、無願解脫爲所入門，無量功德衆所莊嚴，
大寶花王衆所建立大宮殿中。
是薄伽梵最淸淨覺，不二現行，趣無相法。住於佛住逮得一切佛平等性，到無障處，
不可轉法，所行無㝵，其所成立不可思議。遊於三世平等法性，其身流布一切世界，
於一切法智無疑滯，於一切行成就大覺，於諸法智無有疑惑，凡所現身不可分別，
一切菩薩正所求智，得佛無二住勝彼岸，不相閒雜。如來解脫妙智究竟，
證無中邊佛地平等，極於法界，盡虛空性窮未來際。
與無量大聲聞衆俱，一切調順，皆是佛子，心善解脫，慧善解脫，戒善淸淨，趣求法樂；
多聞、聞持，其聞積集；善思所思，善說所說，善作所作；
捷慧、速慧、利慧、出慧、勝決擇慧、大慧、廣慧、及無等慧，慧寶成就；
具足三明，逮得第一現法樂住；大淨福田，威儀寂靜，無不圓滿；
大忍柔和，成就無減，己善奉行如來聖教。
'''

# 3. 전처리: 문장 단위 분리 및 자단위 토큰화
sentences = re.split(r'[。！？；,，\n]', text)  # 마침표/쉼표/줄바꿈 포함
sentences = [s.strip() for s in sentences if s.strip()]

# 자 단위 토큰화
tokenized_sentences = [[char for char in sentence] for sentence in sentences]

print("토큰화된 문장 수:", len(tokenized_sentences))
print("예시:", tokenized_sentences[0])

# 4. Word2Vec 모델 훈련
# CBOW 모델
model_cbow = Word2Vec(sentences=tokenized_sentences, sg=0, vector_size=50, window=3, min_count=1)

# Skip-gram 모델
model_sg = Word2Vec(sentences=tokenized_sentences, sg=1, vector_size=50, window=3, min_count=1)

# 5. 유사도 확인: '如來'를 중심 단어로 테스트
print("\n▼ CBOW 모델 유사 단어 ('如'):")
print(model_cbow.wv.most_similar('如'))

print("\n▼ Skip-gram 모델 유사 단어 ('如'):")
print(model_sg.wv.most_similar('如'))

토큰화된 문장 수: 68
예시: ['一', '時']

▼ CBOW 모델 유사 단어 ('如'):
[('路', 0.32674485445022583), ('切', 0.31467321515083313), ('之', 0.29925477504730225), ('宮', 0.2858406901359558), ('有', 0.27146774530410767), ('照', 0.2571389675140381), ('過', 0.2504095733165741), ('天', 0.24158146977424622), ('虛', 0.23034782707691193), ('從', 0.2146180272102356)]

▼ Skip-gram 모델 유사 단어 ('如'):
[('路', 0.3258456587791443), ('切', 0.31270816922187805), ('之', 0.299013614654541), ('宮', 0.28103673458099365), ('有', 0.27027228474617004), ('照', 0.2542853057384491), ('過', 0.24790138006210327), ('天', 0.239946186542511), ('虛', 0.2333383709192276), ('從', 0.2156377136707306)]


의미적 유사성 평가

‘如’가 ‘路’, ‘切’, ‘之’ 등과 유사하다고 판단된 것은 이 문자들이 본문 내에서 같은 문맥이나 인접 위치에서 자주 등장했기 때문. 예를 들어 “如來所都…大念、慧、行以爲遊路” 같은 문장에서 ‘如’와 ‘路’가 함께 출현하여 공동 등장 빈도에 기반한 유사성 학습이 이뤄진 것으로 해석 가능.
Word2Vec의 핵심 메커니즘은 주변 단어들의 분포적 특성을 학습하는 것이므로, 자소 단위에서 이러한 연관이 학습된 것은 구조적으로 자연스러운 결과임.

현재 문제점

1. 자소(한 글자) 단위 토큰화:
‘如來’, ‘菩薩’, ‘薄伽梵’ 등 복합 불교 개념이 분해되어 의미 손실 발생. 단자(單字) 단위 유사도는 문맥 기반 유사성과는 무관한 결과를 낳기 쉬움.


개선 방법

1. 어절 기반 토큰화:
‘如來’, ‘大菩薩’, ‘無相法’ 등 복합 개념을 하나의 토큰으로 유지해야 의미 보존 가능. 불교 용어 사전을 활용하거나 2~3글자 N-gram을 활용해 세분화 가능.
